In [1]:
from src.preprocessing import *
from src.master import *
from src.modelisation import *
from src.impute import *

pd.options.display.max_columns = None

In [2]:
df = master_preprocessing_X('data/Airport_Data.csv',
                      'data/Weather_data_prep.csv',
                      'data/ACchar.xlsx',
                      'Correspondance.pkl')



//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df['target'] = create_target(df)

df = impute_chocks_on(df)

df = df.dropna()

In [8]:
variables = ['carrier','runway','stand','Manufacturer','acType','PRCP','TAVG','AWND','TMAX','TMIN','WDF2','WDF5','WSF2','WSF5','WT01','WT02','WT03','WT08','Approach Speed\n(Vref)']

encoded_df = encoding_df(df, variables)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn import tree

# Define input 
X = encoding_df(df, variables)

# Set validation
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, y_train)

print(compute_mape(y_test, clf.predict(X_test)))
print(compute_rmse(y_test, clf.predict(X_test)))

y_pred = clf.predict(X_test)

0.59135868190418
5.99414442370663


In [ ]:
def master_modelisation(X,y,test_size,model_function_list) :
    """
    function aggregating models and building comparison table

    input: how many models, training features, target values,
    output : predicted values and rmse

    """    
    models = model_function_list
    
    output = pd.DataFrame()
    model = []
    RMSE = []
    MAPE = []
    tenth_perc = []    
    ninetieth_perc = []
    
    for i in models:
        output_row = pd.DataFrame()
        
        model.append(i)
        
        y_pred, y_test = i(df, variables, test_size)

        RMSE.append(compute_rmse(y_test, y_pred))
        MAPE.append(compute_mape(y_test, y_pred))
        tenth_perc.append(tenth_percentile(y_test, y_pred))   
        ninetieth_perc.append(ninetieth_percentile(y_test, y_pred))
        
    output['Model'] = model
    output['RMSE'] = RMSE
    output['MAPE'] = MAPE
    output['tenth_perc'] = tenth_perc
    output['ninetieth_perc'] = ninetieth_perc
        
    return output

# TEST
master_modelisation(X,y,0.10,[linear_reg,decision_tree])

In [34]:
def compute_metrics(y_test, y_pred) : 
    metrics = pd.DataFrame()
    metrics['RMSE'] = [compute_rmse(y_test, y_pred)]
    metrics['MAPE'] = [compute_mape(y_test, y_pred)]
    metrics['MAE'] = [compute_mae(y_test, y_pred)]
    metrics['tenth_perc'] = [tenth_percentile(y_test, y_pred)]
    metrics['ninetieth_perc'] = [ninetieth_percentile(y_test, y_pred)]        
    
    return metrics

In [27]:
df.target

0         240.0
1         240.0
2         180.0
3         240.0
4         300.0
5         600.0
6         360.0
7         420.0
8         420.0
9         360.0
10        360.0
11        420.0
12        240.0
13        240.0
15        360.0
16        840.0
17        300.0
18        420.0
19        540.0
20        600.0
21        300.0
22        240.0
23        360.0
24        540.0
25        420.0
26        300.0
27        300.0
28        480.0
29        360.0
30        480.0
          ...  
137957    360.0
137959    600.0
137960    480.0
137962    840.0
137964    480.0
137966    420.0
137968    480.0
137969    360.0
137970    480.0
137971    540.0
137972    600.0
137973    360.0
137974    720.0
137975    360.0
137976    540.0
137977    360.0
137978    540.0
137979    480.0
137980    240.0
137981    480.0
137982    360.0
137983    300.0
137984    420.0
137985    840.0
137986    420.0
137987    480.0
137988    420.0
137989    900.0
137990    480.0
137991    420.0
Name: target, Length: 13

In [46]:
def custom_made_metric(y, y_pred, threshold):
    """
    function computing MAE 

    input: real target values, predicted values, threshold (in min) to consider harmfull error
    output : root mean squared error in minutes
    """
    error = (y - y_pred)
    metric = (error > threshold*60)
    return metric.mean()


In [48]:
compute_metrics(y_test,y_pred)

,RMSE,MAPE,MAE,tenth_perc,ninetieth_perc
0,5.994144,0.591359,3.066873,0.0,420.0
